# Mission to Mars (Data Scraping)

In [1]:
# All scraping and analysis
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import pymongo
import requests
import time

In [2]:
#url list
url_mars_news = 'https://mars.nasa.gov/news/'
url_jpl_image = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
url_mars_weather = 'https://twitter.com/marswxreport?lang=en'
url_mars_facts = 'https://space-facts.com/mars/'
url_hemispheres = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'


## Mars News

In [3]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

browser.visit(url_mars_news)

time.sleep(1)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

news_title = soup.find('div', class_="list_text").find('div', class_="content_title").get_text(strip=True)
news_p = soup.find('div', class_="article_teaser_body").get_text(strip=True)

print(news_title)
print('-------------------')
print(news_p)

browser.quit()

NASA's MAVEN Observes Martian Night Sky Pulsing in Ultraviolet Light
-------------------
Vast areas of the Martian night sky pulse in ultraviolet light, according to images from NASA’s MAVEN spacecraft. The results are being used to illuminate complex circulation patterns in the Martian atmosphere.


## JPL Featured Space Image

In [4]:
image_root = 'https://www.jpl.nasa.gov'

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

browser.visit(url_jpl_image)

time.sleep(1)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

jpg_path = soup.find('article', class_="carousel_item")["style"]

# print(jpg_path)
# print(jpg_path.find('url'))
# print(jpg_path[23:])
featured_image_url = image_root + jpg_path[jpg_path.find('url')+5:jpg_path.find('jpg')+3]
# print(jpg_path[jpg_path.find('url')+5:jpg_path.find('jpg')+3])
print(featured_image_url)

browser.quit()

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18816-1920x1200.jpg


## Weather

In [7]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

browser.visit(url_mars_weather)

time.sleep(1)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

weather_tweet = soup.find('section', class_="css-1dbjc4n").find('span', class_="css-901oao")

print(weather_tweet)

browser.quit()


<span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"><span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">Mars Weather</span></span>


## Facts

## Hemisphere Photos